
# metrics.py API & Algorithm Documentation

***Every metric has the same basic interface:***
```python
class Metric(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def reset_parameters(self, parameters):
        return

    @abc.abstractmethod
    def process_data(self, data):
        return

    @abc.abstractmethod
    def get_statistic(self):
        return

    @abc.abstractmethod
    def get_all_statistics(self):
        return
```

## Setup

In [ ]:
from evo.core import metrics

**...some additional modules and settings for this demo:**

In [ ]:
from evo.tools import log
log.configure_logging(verbose=True, debug=True, silent=False)

import pprint
import numpy as np

from evo.tools import plot
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False

**Load two example trajectory files in TUM format;**

In [ ]:
from evo.tools import file_interface

ref_file = "../test/data/freiburg1_xyz-groundtruth.txt"
est_file = "../test/data/freiburg1_xyz-rgbdslam_drift.txt"

traj_ref = file_interface.read_tum_trajectory_file(ref_file)
traj_est = file_interface.read_tum_trajectory_file(est_file)

 **The metrics require the trajectories to be associated via matching timestamps:**

In [ ]:
from evo.core import sync

max_diff = 0.01

traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est)

**Optionally, trajectory points can be aligned. Since we know the data associations, the alignment can be calculated in closed form using Umeyama's method. Additionally / alternatively, the scale can be corrected (e.g. for monocular SLAM).**

In [ ]:
from evo.core import trajectory

traj_est_aligned = trajectory.align_trajectory(traj_est, traj_ref, correct_scale=False, correct_only_scale=False)

**Plot the trajectories:**

In [ ]:
fig = plt.figure()
traj_by_label = {
    "estimate (not aligned)": traj_est,
    "estimate (aligned)": traj_est_aligned,
    "reference": traj_ref
}
plot.trajectories(fig, traj_by_label, plot.PlotMode.xyz)
plt.show()

<a id='ape_math'></a>
## APE
***The absolute pose error is a metric for investigating the global consistency of a SLAM trajectory***

APE is based on the absolute relative pose between two poses $P_{ref,i}, P_{est,i} \in \mathrm{SE}(3)$ at timestamp $i$:
\begin{equation*}
E_i = P_{ref,i} \ominus P_{est,i} = P_{ref,i}^{-1} P_{est,i} \in \mathrm{SE}(3)
\end{equation*}
You can use different pose relations to calculate the APE:
* **`metrics.PoseRelation.translation_part`**
    * this uses the translation part of $E_i$
    * $ APE_i = \| \mathrm{trans}(E_i) \| $
* **`metrics.PoseRelation.rotation_angle_(rad/deg)`**
    * uses the rotation angle of $E_i$
    * $ APE_i = |( \mathrm{angle}(\log_{\mathrm{SO}(3)}(\mathrm{rot}(E_i)) )| $
    * $ \log_{\mathrm{SO}(3)}(\cdot) $ is the inverse of $ \exp_{\mathfrak{so}(3)}(\cdot) $ (Rodrigues' formula)
* **`metrics.PoseRelation.rotation_part`**
    * this uses the rotation part of $E_i$
    * $ APE_i = \| \mathrm{rot}(E_i) - I_{3 \times 3} \|_F $
* **`metrics.PoseRelation.full_transformation`**
    * this uses the full relative pose $E_i$
    * $ APE_i = \| E_i - I_{4 \times 4} \|_F $
    
Then, different statistics can be calculated on the APEs of all timestamps, e.g. the RMSE:
\begin{equation*}
\mathrm{RMSE} = \sqrt{ \frac{1}{N} \sum_{i=1}^N APE_i^2 } 
\end{equation*}

### Settings

In [ ]:
pose_relation = metrics.PoseRelation.translation_part
use_aligned_trajectories = False

### Data Preparation
Optionally, we can use the aligned trajectory:

In [ ]:
if use_aligned_trajectories:
    data = (traj_ref, traj_est_aligned) 
else:
    data = (traj_ref, traj_est)

### Run APE on Data
Create an instance of the APE class and process the data

In [ ]:
ape_metric = metrics.APE(pose_relation)
ape_metric.process_data(data)

### Get APE Statistics
Get a single statistic:

In [ ]:
ape_stat = ape_metric.get_statistic(metrics.StatisticsType.rmse)
print(ape_stat)

Get all avalaible statistics at once in a dictionary:

In [ ]:
ape_stats = ape_metric.get_all_statistics()
pprint.pprint(ape_stats)

Plot the APE values and statistics:

In [ ]:
seconds_from_start = [t - traj_est.timestamps[0] for t in traj_est.timestamps]
fig = plt.figure()
plot.error_array(fig, ape_metric.error, x_array=seconds_from_start, statistics=ape_stats, 
                 name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
plt.show()

Plot the trajectory with colormapping of the APE:

In [ ]:
plot_mode = plot.PlotMode.xy
fig = plt.figure()
ax = plot.prepare_axis(fig, plot_mode)
plot.traj(ax, plot_mode, traj_ref, '--', "gray", "reference")
plot.traj_colormap(ax, traj_est_aligned if use_aligned_trajectories else traj_est, ape_metric.error, 
                   plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
ax.legend()
plt.show()

### Comparison with TUM ATE script

The official TUM RGB-D benchmark script `evaluate_ate.py` gives the RMSE of the xyz difference of the aligned trajectories:

In [ ]:
import subprocess as sp
import sys
cmd = ["python2", "../test/tum_benchmark_tools/evaluate_ate.py", ref_file, est_file, "--max_difference", str(max_diff)]
out = sp.check_output(cmd)
print(out.decode(sys.stdout.encoding))

...which is equivalent to:

In [ ]:
tum_ate_equivalent = metrics.APE(metrics.PoseRelation.translation_part)
tum_ate_equivalent.process_data((traj_ref, traj_est_aligned))
print(tum_ate_equivalent.get_statistic(metrics.StatisticsType.rmse))

<a id='rpe_math'></a>
# RPE

***The relative pose error is a metric for investigating the local consistency of a SLAM trajectory***

RPE compares the relative poses along the estimated and the reference trajectory. This is based on the delta pose difference: 
\begin{equation*} E_{i,j} = \delta_{ref_{i,j}} \ominus \delta_{est_{i,j}} = (P_{ref,i}^{-1}P_{ref,j})^{-1} (P_{ref,i}^{-1}P_{est,j}) \in \mathrm{SE}(3) \end{equation*}

You can use different pose relations to calculate the RPE from timestamp $i$ to $j$:
* **`metrics.PoseRelation.translation_part`**
    * this uses the translation part of $E_{i,j}$
    * $ RPE_{i,j} = \| \mathrm{trans}(E_{i,j}) \| $
* **`metrics.PoseRelation.rotation_angle_(rad/deg)`**
    * uses the absolute angular error of $E_{i,j}$
    * $ RPE_{i,j} = |( \mathrm{angle}(\log_{\mathrm{SO}(3)}(\mathrm{rot}(E_{i,j})) )| $
    * $ \log_{\mathrm{SO}(3)}(\cdot) $ is the inverse of $ \exp_{\mathfrak{so}(3)}(\cdot) $ (Rodrigues' formula)
* **`metrics.PoseRelation.rotation_part`**
    * this uses the translation part of $E_{i,j}$
    * $ RPE_{i,j} = \| \mathrm{rot}(E_{i,j}) - I_{3 \times 3} \|_F $
* **`metrics.PoseRelation.full_transformation`**
    * this uses the full delta pose difference $E_{i,j}$
    * $ RPE_{i,j} = \| E_{i,j} - I_{3 \times 3} \|_F $
    
Then, different statistics can be calculated on the RPEs of all timestamps, e.g. the RMSE:
\begin{equation*}
\mathrm{RMSE} = \sqrt{ \frac{1}{N} \sum_{\forall ~i,j} RPE_{i,j}^2 } 
\end{equation*}

### Settings

The parameter $\Delta$ determines the distance between the pose pairs along the trajectories. E.g. if you have 30 poses per second and want to measure the RPE every second, use $\Delta=30 ~\text{(frames)}$. Or to measure everytime you moved 1 meter, $\Delta=1 ~\text{(m)}$, ...

Another option is to use all possible pairs of a certain delta value, i.e. not only the subsequent (linear) delta pairs of the trajectory.

Using aligned trajectories does not make sense with RPE because the delta poses are the same as with the unaligned case.

In [ ]:
pose_relation = metrics.PoseRelation.rotation_angle_deg

# normal mode
delta = 1
delta_unit = metrics.Unit.frames

# all pairs mode
all_pairs = False  # activate

### Data Preparation

In [ ]:
data = (traj_ref, traj_est)

### Run RPE on Data
Create an instance of the RPE class and process the data

In [ ]:
rpe_metric = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
rpe_metric.process_data(data)

### Get RPE Statistics
Get a single statistic:

In [ ]:
rpe_stat = rpe_metric.get_statistic(metrics.StatisticsType.rmse)
print(rpe_stat)

Get all avalaible statistics at once in a dictionary:

In [ ]:
rpe_stats = rpe_metric.get_all_statistics()
pprint.pprint(rpe_stats)

**Plot the RPE values and statistics:**

In [ ]:
# important: restrict data to delta ids for plot
import copy
traj_ref_plot = copy.deepcopy(traj_ref)
traj_est_plot = copy.deepcopy(traj_est)
traj_ref_plot.reduce_to_ids(rpe_metric.delta_ids)
traj_est_plot.reduce_to_ids(rpe_metric.delta_ids)
seconds_from_start = [t - traj_est.timestamps[0] for t in traj_est.timestamps[1:]]

In [ ]:
fig = plt.figure()
plot.error_array(fig, rpe_metric.error, x_array=seconds_from_start, statistics=rpe_stats, 
                 name="RPE", title="RPE w.r.t. " + rpe_metric.pose_relation.value, xlabel="$t$ (s)")
plt.show()

Plot the trajectory with colormapping of the RPE:

In [ ]:
plot_mode = plot.PlotMode.xy
fig = plt.figure()
ax = plot.prepare_axis(fig, plot_mode)
plot.traj(ax, plot_mode, traj_ref_plot, '--', "gray", "reference")
plot.traj_colormap(ax, traj_est_plot, rpe_metric.error, plot_mode, min_map=rpe_stats["min"], max_map=rpe_stats["max"])
ax.legend()
plt.show()

### Comparison with TUM RPE script

The official TUM RGB-D benchmark script evaluate_rpe.py uses all possible pairs (in "fixed_delta" mode only!) of the specified delta. 

The default output is the mean of the RPE:

In [ ]:
import subprocess as sp
delta = 15
cmd = cmd = ["python2", "../test/tum_benchmark_tools/evaluate_rpe.py", ref_file, est_file, "--delta", str(delta), "--delta_unit", 'f', '--fixed_delta']
out = sp.check_output(cmd)
print(out.decode(sys.stdout.encoding))

...which is equivalent to:

In [ ]:
tum_rpe_equivalent = metrics.RPE(metrics.PoseRelation.translation_part, delta, metrics.Unit.frames, all_pairs=True)
tum_rpe_equivalent.process_data((traj_ref, traj_est))
print(tum_rpe_equivalent.get_statistic(metrics.StatisticsType.mean))